# Word/Character Level Generation Using LSTM

Hii Folks...

In This Notebook We are going to work on Character And Word Level Generation. Basically I Recently learn About LSTM And I want to create something using Them And I Found That Word Or Character Level Generation is such an Excellend Project.

So Before The Project Let's talk about few things-

* **What are RNN's?**
* **Why RNN networks Fail?**
* **LSTM**



### What Are RNN's 
Human's Don't start there thinking From Scratch every Second.
For Example-> *"He Like going to School."*
You are not able to guess What I Am Talking About? *"Who's He"*.
Right But If I Give you Statement Like *"Alex is good in acaedmics. He Like going to school"*.
You Can Understand I Am talking about *Alex* Earlier.

Recurrent Neural Networks address this issue. They are networks with loops in them, allowing information to persist.

![Rnn Image](https://media.geeksforgeeks.org/wp-content/uploads/20231204125839/What-is-Recurrent-Neural-Network-660.webp)

They are Like Networks with loop. If they have a information Say A Sentence *"This Cat is Big."*. Then Our RNN is first going to take the first word *The* and then is Just Like A Normal ANN where our input is going to multiply with weighs and adding of biases and then we are going to return our output but RNN also send that output to Next ANN and Now it is Going to take two input First *"Cat"* from sentence and our answer for *The* and now it is going to give response based on those two words. and It is Going to Continue until the sentence end.

**Note - We are also going to convert our words to vectors. As Input Layer Takes Numbers.**

This chain-like nature reveals that recurrent neural networks are intimately related to sequences and lists. They’re the natural architecture of neural network to use for such data.
RNN work output is dependent on the earlier context that is the reason they become so famous. Many Problems can be solved using RNN's. They solved varity of problems speech recognition, language modeling, translation, image captioning… and the list goes on.  But they really are pretty amazing.

They are also Building Blocks Of **LSTM** as well.

Now main Question is So Why they need LSTM??


### Why RNN networks Fail?

One of the main reason to RNN. That they can able to connect previous info and current task. For Example. It can use previous stock market data  to help us in making a better understanding of current market data.
But Can They? It actually Depends.

It depends on how previous info we are looking. For example, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in “Lions are  king of Jungle” we don’t need any further context – it’s pretty obvious the next word is going to be Jungle. In such cases, where the gap between the relevant information and the place that it’s needed is small, RNNs can learn to use the past information.


But there are also cases where we need more context. Consider trying to predict the last word in the text “His Name is Jonathan… He speak fluent French.” Recent information suggests that the next word is probably the name of a language, but if we want to narrow down which language, we need the context of Jonathan, from further back. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large.

Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

It happends due to **Vanishing Gradient Problem.** 

Thankfully, LSTMs don’t have this problem!

## **LSTM**

Long Term Short Memory or LSTM Are Special Kind of RNN. They are specially created to avoid long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!

In concept, an LSTM recurrent unit tries to “remember” all the past knowledge that the network is seen so far and to “forget” irrelevant data. This is done by introducing different activation function layers called “gates” for different purposes. Each LSTM recurrent unit also maintains a vector called the Internal Cell State which conceptually describes the information that was chosen to be retained by the previous LSTM recurrent unit.

LSTM networks are the most commonly used variation of Recurrent Neural Networks (RNNs). The critical component of the LSTM is the memory cell and the gates (including the forget gate but also the input gate), inner contents of the memory cell are modulated by the input gates and forget gates. This allows the LSTM model to overcome the vanishing gradient properly occurs with most Recurrent Neural Network models.



## Importing The Libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import string
import random
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalAveragePooling1D,Input
from tensorflow.keras.models import Sequential
import re

### Word Level Text Generation
Firstly I Am going to use word level text generation where our model is going to generate word For The Input Text.

In [ ]:
strategy = tf.distribute.MirroredStrategy()

## Importing Data

In [ ]:
data=pd.read_csv("/kaggle/input/shakespeare-plays/Shakespeare_data.csv")
data.head()

In [ ]:
dataset=data["PlayerLine"]

In [ ]:
corpus=[]
with strategy.scope():
    for line in dataset:
        lowercase_line=line.lower()
        corpus.append(lowercase_line)
    

In [ ]:
corpus[:10]

## Converting Word to integers

We are going to Create Tokenizer and Going to fit on our texts. Converting all our words to an index

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
word_to_token=tokenizer.word_index
index_or_word=tokenizer.index_word

In [ ]:
def check_dict(num):
    for i in range(num):
        numrand=random.randrange(0,len(index_or_word)-1)
        print(f"Word- {index_or_word[numrand]} and index- {numrand}")
    
check_dict(10)

In [ ]:
total_words=len(word_to_token)+1
total_words

In [ ]:
input_sequences=[]

with strategy.scope():
    c=0
    for line in corpus:
        token_list=tokenizer.texts_to_sequences([line])[0]
        for i in range(1,len(token_list)):
            n_gram_token=token_list[:i+1]
            input_sequences.append(n_gram_token)

In [ ]:
len(input_sequences)

In [ ]:
before=input_sequences[0]
before

In [ ]:
max_seq_len=max(len(x) for x in input_sequences)

In [ ]:
max_seq_len

All the texts Should be of same length. So We are going to do predadding or adding zero. So each line going to be of same length.


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_sequences_array = np.array(pad_sequences(input_sequences,
                                              maxlen=max_seq_len, 
                                              padding = 'pre'))

In [ ]:
features=input_sequences_array[:,:-1]

In [ ]:
labels=input_sequences_array[:,-1]

In [ ]:
labels=tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [ ]:
labels[0]

In [ ]:
with strategy.scope():
    n = 0.5# We are only taking a chunk of this huge dataset to fit it on the RAM
    slice_size = int(len(features)*n)
    np.save('/kaggle/working/features', features[:slice_size, :])
    np.save('/kaggle/working/labels', labels[:slice_size, :])


In [ ]:
with strategy.scope():
    features = np.load('/kaggle/working/features_new.npy')
    labels = np.load('/kaggle/working/labels_new.npy')

In [ ]:
total_words

### Creating Dataset for word level text Generation

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

# Optional: Shuffle and batch the dataset
BATCH_SIZE = 32
dataset = dataset.shuffle(buffer_size=len(features)).batch(BATCH_SIZE)

# Verify the dataset
for batch_features, batch_labels in dataset.take(1):
    print("Batch Features Shape:", batch_features.shape)
    print("Batch Labels Shape:", batch_labels.shape)
    

## Creating Model 

In [ ]:
def generate_model():
    tf.random.set_seed(42)
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_seq_len-1))  # Corrected
    model.add(Bidirectional(LSTM(64, return_sequences=True)))            # Corrected
    model.add(Bidirectional(LSTM(32)))                                   # Corrected
    model.add(Dense(64, activation='relu'))                              # Corrected
    model.add(Dense(total_words, activation='softmax'))                  # Corrected
    return model


In [ ]:
with strategy.scope():
    model = generate_model()
    model.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
                  metrics=['accuracy'])
    model.build(input_shape=(None, max_seq_len-1))
    model.summary()
    

In [ ]:
EPOCHS = 50
history = model.fit(features, labels, epochs = EPOCHS)

In [ ]:
def plot_graph(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [ ]:
plot_graph(history, 'accuracy')
plot_graph(history, 'loss')

### Saving The Word Level Model

In [ ]:
model.save('/kaggle/working/test_generator_model.h5')

### Testing Our Model

In [ ]:
def test_generator(string, num):
    if len(string)==0:
        print("Error: No word found")
        return
    for _ in range(num):
        token_list = tokenizer.texts_to_sequences([string])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding = "pre")
        probabilities = model.predict(token_list)
        choice = np.random.choice([1,2,3])
        predicted = np.argsort(probabilities, axis = -1)[0][-choice]
        if predicted !=0:
            generated_word = tokenizer.index_word[predicted]
            string += " " + generated_word
    print(string)

In [ ]:
max_seq_len

In [ ]:
test_generator("Enter KING HENRY", 20)


## Character Level Text Generation

We are going to Create A Model Which is Going to generate *characters.* 

For this I Am Going to use *nietzsche works*

In [ ]:
!curl -O https://s3.amazonaws.com/text-datasets/nietzsche.txt

In [ ]:
with open("/kaggle/working/nietzsche.txt","r") as file:
    text=file.read()

## Creating Dataset

In [ ]:
text[:10]

In [ ]:
batch_size = 64
raw_data_ds = tf.data.TextLineDataset(["nietzsche.txt"])

In [ ]:
for elem in raw_data_ds.take(10):
    print(elem.numpy().decode("utf-8"))

In [ ]:
text=""

for elem in raw_data_ds:
    text+=elem.numpy().decode('utf-8')

In [ ]:
len(text)


In [ ]:
chars=sorted(list(set(text)))
print("Total disctinct chars:", len(chars))


In [ ]:
maxlen = 20
step = 3
input_chars = [] # X
next_char = []   # y

In [ ]:
for i in range(0,len(text)-maxlen,step):
    input_chars.append(text[i:i+maxlen])
    next_char.append(text[i + maxlen])

In [ ]:
input_chars[1],next_char[1]

In [ ]:
print("Number of sequences:", len(input_chars))
print("input X  (input_chars)  --->   output y (next_char) ")
for i in range(8):
      print( input_chars[i],"   --->  ", next_char[i])

### Using Tensorflow Slices

In [ ]:
X_train_ds_raw=tf.data.Dataset.from_tensor_slices(input_chars)
y_train_ds_raw=tf.data.Dataset.from_tensor_slices(next_char)

In [ ]:
for elem1,elem2 in zip(X_train_ds_raw.take(5),y_train_ds_raw.take(5)):
    print(elem1.numpy().decode('utf-8'),"----->", elem2.numpy().decode('utf-8'))

In [ ]:
# Model constants.
max_features = 83              # Number of distinct chars  
embedding_dim = 16             # Embedding layer output dimension
sequence_length = maxlen 

### Text Preprocessing

In [ ]:
def custom_standardization(input_data):
    lowercase     = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    stripped_num  = tf.strings.regex_replace(stripped_html, "[\d-]", " ")
    stripped_punc  =tf.strings.regex_replace(stripped_num, 
                             "[%s]" % re.escape(string.punctuation), "")    
    return stripped_punc

def char_split(input_data):
    
    return tf.strings.unicode_split(input_data, 'UTF-8')

### Creating Text Vectorization Layer

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,      # Preprocessing function
    max_tokens= max_features,                # Maximum size of the vocabulary for this layer     
    split=char_split,                        # Split into chars
    output_mode="int",                       # convert to int
    output_sequence_length=sequence_length,  # Training sample length
)

In [ ]:
vectorize_layer.adapt(X_train_ds_raw.batch(batch_size))

In [ ]:
print("The first 10 entries: ", vectorize_layer.get_vocabulary()[:10])

In [ ]:
for i in X_train_ds_raw.take(2):
    print(i.numpy().decode("utf-8"))

In [ ]:
def vectorize_text(text):
    text = tf.expand_dims(text, -1)
    return tf.squeeze(vectorize_layer(text))

In [ ]:
X_train_ds = X_train_ds_raw.map(vectorize_text)
y_train_ds = y_train_ds_raw.map(vectorize_text)

X_train_ds.element_spec, y_train_ds.element_spec

In [ ]:
for text in X_train_ds.take(1):
    print(text.numpy())
for text in y_train_ds.take(1):
    print(text.numpy())

In [ ]:
y_train_ds=y_train_ds.map(lambda x: x[0])

In [ ]:
for elem in y_train_ds.take(1):
    print("shape: ", elem.shape, "\n next_char: ",elem.numpy())

In [ ]:
train_ds =  tf.data.Dataset.zip((X_train_ds,y_train_ds))

In [ ]:
for (X,y) in zip(X_train_ds.take(5), y_train_ds.take(5)):
    print(X.numpy()," --> ",y.numpy())

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(buffer_size=512).batch(batch_size, drop_remainder=True).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for sample in train_ds.take(1):
    print("input (X) dimension: ", sample[0].numpy().shape, "\noutput (y) dimension: ",sample[1].numpy().shape)

In [ ]:
for sample in train_ds.take(1):
  print("input (sequence of chars): ", sample[0][0].numpy(), "\noutput (next char to complete the input): ",sample[1][0].numpy())

In [ ]:
def decode_sequence(encoded_sequence):
    
    deceoded_sequence=[]
    for token in encoded_sequence:
        deceoded_sequence.append(vectorize_layer.get_vocabulary()[token])
    sequence= ''.join(deceoded_sequence)
    print("\n",sequence,"\n")
    return sequence

In [ ]:
for sample in train_ds.take(7):
     print("input (sequence of chars): ", 
           decode_sequence (sample[0][0].numpy()), 
           "\noutput (next char to complete the input): ",
           vectorize_layer.get_vocabulary()[sample[1][0].numpy()]
          )

In [ ]:
def temperature_sampling (conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

In [ ]:
def top_k_sampling(conditional_probability, k):
    top_k_probabilities, top_k_indices= tf.math.top_k(conditional_probability, k=k, sorted=True)
    top_k_probabilities= np.asarray(top_k_probabilities).astype("float32")
    top_k_probabilities= np.squeeze(top_k_probabilities)
    top_k_indices = np.asarray(top_k_indices).astype("int32")
    top_k_redistributed_probability=softmax(top_k_probabilities)
    top_k_redistributed_probability = np.asarray(top_k_redistributed_probability).astype("float32")
    sampled_token = np.random.choice(np.squeeze(top_k_indices), p=top_k_redistributed_probability)
    return sampled_token

In [ ]:
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K


### Creating Character Level Model

In [ ]:
inputs = tf.keras.Input(shape=(sequence_length,), dtype="int64")
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(128, return_sequences=True)(x)
x = layers.LSTM(128, return_sequences=False)(x)
predictions=  layers.Dense(max_features, activation='softmax')(x)
model_LSTM = tf.keras.Model(inputs, predictions,name="model_LSTM")

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=5)
model_LSTM.compile(loss='sparse_categorical_crossentropy', 
                   optimizer='adam', metrics=['accuracy'])
print(model_LSTM.summary())

In [ ]:
vectorize_layer("Hii I Am")

In [ ]:
history=model_LSTM.fit(train_ds,epochs=100,callbacks=[callback])

### Plotiing Accuracy And Loss

In [ ]:
plot_graph(history,"accuracy")
plot_graph(history,"loss")

In [ ]:
def greedy_search(conditional_probability):
    
    return (np.argmax(conditional_probability))

In [ ]:
def softmax(z):
     return np.exp(z)/sum(np.exp(z))
    
def temperature_sampling (conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

In [ ]:
# using only the top K props as distribution
def top_k_sampling(conditional_probability, k):
    top_k_probabilities, top_k_indices= tf.math.top_k(conditional_probability, k=k, sorted=True)
    top_k_probabilities= np.asarray(top_k_probabilities).astype("float32")
    top_k_probabilities= np.squeeze(top_k_probabilities)
    top_k_indices = np.asarray(top_k_indices).astype("int32")
    top_k_redistributed_probability=softmax(top_k_probabilities)
    top_k_redistributed_probability = np.asarray(top_k_redistributed_probability).astype("float32")
    sampled_token = np.random.choice(np.squeeze(top_k_indices), p=top_k_redistributed_probability)
    return sampled_token

### Testing Out Our Model.
We are going to use three sample techniques  **Top K sampling**, **Temperature Sampling** and **Greedy Search**
We are going to predict our next character based on these. 

So How these Algorithms Work-
* Top K Sampling - It does exactly what it sounds it is going to selects the most k probable words or characters.
* Greedy Search- It is Simple. It is going to select word or character with highest probability.
* Temperature Sampling- You will set a temperature from 0 to more than 1. The More you closer to 0 the words with high  probability only get chances but when as you go towards more distant words like you set temperature of 1.5. Model Also pick random words or words with less probability

### Testing Our Model

In [ ]:
def generate_text(model, seed_original, step):
    seed= vectorize_text(seed_original)
    print("The prompt is")
    decode_sequence(seed.numpy().squeeze())
    

    seed= vectorize_text(seed_original).numpy().reshape(1,-1)
    #Text Generated by Greedy Search Sampling
    generated_greedy_search = (seed)
    for i in range(step):
        predictions=model.predict(seed)
        next_index= greedy_search(predictions.squeeze())
        generated_greedy_search = np.append(generated_greedy_search, next_index)
        seed= generated_greedy_search[-sequence_length:].reshape(1,sequence_length)
    print("Text Generated by Greedy Search Sampling:")
    decode_sequence(generated_greedy_search)

    #Text Generated by Temperature Sampling
    print("Text Generated by Temperature Sampling:")
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print("\ttemperature: ", temperature)
        seed= vectorize_text(seed_original).numpy().reshape(1,-1)
        generated_temperature = (seed)
        for i in range(step):
            predictions=model.predict(seed)
            next_index = temperature_sampling(predictions.squeeze(), temperature)
            generated_temperature = np.append(generated_temperature, next_index)
            seed= generated_temperature[-sequence_length:].reshape(1,sequence_length)
        decode_sequence(generated_temperature)

    #Text Generated by Top-K Sampling
    print("Text Generated by Top-K Sampling:")
    for k in [2, 3, 4, 5]:
        print("\tTop-k: ", k)
        seed= vectorize_text(seed_original).numpy().reshape(1,-1)
        generated_top_k = (seed)
        for i in range(step):
            predictions=model.predict(seed)
            next_index = top_k_sampling(predictions.squeeze(), k)
            generated_top_k = np.append(generated_top_k, next_index)
            seed= generated_top_k[-sequence_length:].reshape(1,sequence_length)
        decode_sequence(generated_top_k)

In [ ]:
generate_text(model_LSTM,"Truth is a woma", 1) 


In [ ]:
text[:10]

In [ ]:
generate_text(model_LSTM,"is a wo", 5) 

### Saving Our Model

In [ ]:
model_LSTM.save("/kaggle/working/LSTM_Model_new.h5")